In [1]:
import numpy as  np
import pandas as pd
import sklearn
import os
import json
from scipy.spatial.distance import cosine as cosine
from collections import defaultdict

In [2]:
with open("Assignment-4/labeled_x.txt",mode="rb") as f:
    text = f.readlines()
text = [x.strip().decode("utf-8", "ignore")  for x in text]

with open("Assignment-4/labeled_y.txt",mode="rb") as f:
    label = f.readlines()
label = [x.strip().decode("utf-8", "ignore")  for x in label]

with open("Assignment-4/unlabeled_x.txt",mode="rb") as f:
    unlab = f.readlines()
unlab = [x.strip().decode("utf-8", "ignore")  for x in unlab]

print(len(text), len(label), len(unlab))

350 350 1300


In [3]:
# nltk support 


import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

import spacy
nlp = spacy.load("en")

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
stop = set(stopwords.words('english'))

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/anoopkumar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/anoopkumar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import re
fil_text = []
for i in text:
  x = re.sub("(\\W|\\d)"," ", i)
  tokenizer = RegexpTokenizer(r'\w+')
  x = tokenizer.tokenize(i)
  x = [word.lower() for word in x ]
  x = [lemmatizer.lemmatize(word=word,pos='v') for word in x]
  x = [word for word in x if word not in stop]
  x = [word for word in x if word.isalpha()]
  fil_text.append(" ".join(x))

print(len(fil_text))

350


In [7]:
print(text[0])
print(fil_text[0])

The server was really cool and served us our food and drinks with a smile.
server really cool serve us food drink smile


In [170]:
def noun_fetch(stmt):
#     doc = nlp("The server was really cool and served us our food and drinks with a smile.")
    doc = nlp(stmt)
    noun_pharses=set()
    for nc in doc.noun_chunks:
        for np in [nc, doc[nc.root.left_edge.i:nc.root.right_edge.i+1]]:
            if np.text.lower() not in stop:
                noun_pharses.add(np)
    return noun_pharses

In [232]:
# review = []
# catg = []

# for i in text:
#     review.append(list(noun_fetch(i)))
# print(len(review))


print(review[0])

[us, our food, a smile, The server, a smile, us, The server, our food and drinks]


In [23]:
def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel

embeddings_index  = loadGloveModel("/home/anoopkumar/DLNLP/asg2/Assignment-2/glove.42B.300d.txt")

Loading Glove Model
1917494  words loaded!


In [308]:
#aspects

aspects = ["quality","ambience", "place","lights","prices","costly","cost","cheap","location","food","drinks","wine",
           "service","manner","waiter","staff","options"]

aspectsd = {"ambience":["ambience"],
            "location":["location"],
            "prices":["prices","costly","cost","cheap"],
            "quality":["quality","food","drinks","wine"],
            "service":["service","manner","waiter","staff"],
              "options":["options"]}
asemb = []
for i in aspects:
    asemb.append(embeddings_index[i])

    
def getEmb(x):
    y = re.sub("(\\W|\\d)"," ", x)
    tokenizer = RegexpTokenizer(r'\w+')
    x = tokenizer.tokenize(x)
#     x = x.split(" ")
    x = [word for word in x if word.lower() not in stop]
    if len(x) == 0:
        return -2
    if len(x) == 1:
        return embeddings_index[x[0].lower()]
    else:
#         print("hi,",xt)
        xt = np.array([embeddings_index[word.lower()] for word in x if word.lower() in embeddings_index])
        xt = xt.mean(axis = 0)
        return xt
    

In [277]:
cosine(embeddings_index["place"],embeddings_index["location"])

0.3720151562441987

In [309]:
#150 to 200


ctext = review
clab = label
mylab = []


for x in ctext:
    lab = set()
    for i,asp in enumerate(aspectsd):
        for kk in aspectsd[asp]:
            idd = aspects.index(kk)
            tagemb = asemb[idd]
            for y in x:
                yemb = getEmb(y.text.lower())
                cosSim = 1  - cosine(tagemb,yemb)
#                 print(aspects[i], y.text, cosSim)
                if abs(cosSim) > 0.65:
                    lab.add(asp)
    mylab.append(list(lab))
            
    
for i in range(len(review)):
#     print(text[i])
#     print(ctext[i-144])
    print(clab[i])
    print(mylab[i])
    print("\n############################3")

    




# for i in clab:
#     temp = i.split(";")[:-1]
#     temp = [t.split(":")[0] for t in temp]
#     print(temp)

SERVICE:positive;
['quality']

############################3
QUALITY:positive;STYLE_OPTIONS:positive;
[]

############################3
QUALITY:neutral;
[]

############################3
none;
[]

############################3
none;
[]

############################3
QUALITY:positive;
['quality']

############################3
AMBIENCE:positive;
[]

############################3
AMBIENCE:positive;
['ambience']

############################3
QUALITY:positive;
['quality']

############################3
none;
[]

############################3
none;
[]

############################3
none;
[]

############################3
STYLE_OPTIONS:positive;
['quality']

############################3
AMBIENCE:positive;
[]

############################3
none;
[]

############################3
none;
[]

############################3
SERVICE:negative;
['service']

############################3
none;
[]

############################3
QUALITY:positive;
[]

############################3
QUALITY:negative;
[]



In [ ]:
###########################################################################################################3

In [209]:

import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet

from IPython.display import display
lemmatizer = nltk.WordNetLemmatizer()
import matplotlib
matplotlib.use('Agg')

In [213]:
document = 'THe Pizza and wine were excellent-the service too--but what really MADE this place was the backyard dining area.
tokens = [nltk.word_tokenize(sent) for sent in [document]]
# nltk.download('averaged_perceptron_tagger')
postag = [nltk.pos_tag(sent) for sent in tokens][0]
pos_tag([document])

'THe Pizza and wine were excellent-the service too--but what really MADE this place was the backyard dining area.'

In [310]:

grammar = r"""
    NBAR:
        {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns
        {<RB.?>*<VB.?>*<JJ>*<VB.?>+<VB>?} # Verbs and Verb Phrases
        
    NP:
        {<NBAR>}
        {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
        
"""

cp = nltk.RegexpParser(grammar)
tree = cp.parse(postag)
# print(tree)

In [311]:
def leaves(tree):
    """Finds NP (nounphrase) leaf nodes of a chunk tree."""
    for subtree in tree.subtrees(filter = lambda t: t.label() =='NP'):
        yield subtree.leaves()
        
def get_word_postag(word):
    if pos_tag([word])[0][1].startswith('J'):
        return wordnet.ADJ
    if pos_tag([word])[0][1].startswith('V'):
        return wordnet.VERB
    if pos_tag([word])[0][1].startswith('N'):
        return wordnet.NOUN
    else:
        return wordnet.NOUN
    
def normalise(word):
    """Normalises words to lowercase and stems and lemmatizes it."""
    word = word.lower()
    postag = get_word_postag(word)
    word = lemmatizer.lemmatize(word,postag)
    return word

def get_terms(tree):    
    for leaf in leaves(tree):
        terms = [normalise(w) for w,t in leaf]
        yield terms

terms = get_terms(tree)

features = []
for term in terms:
    _term = ''
    for word in term:
        _term += ' ' + word
    features.append(_term.strip())
features

In [ ]:
###########################################################################################

In [317]:

# !pip install aspect-based-sentiment-analysis

In [318]:
print(len(text), len(clab))

350 350


In [17]:
# import aspect_based_sentiment_analysis as absa

# nlp = absa.load()
text = ("THe Pizza and wine were bad-the service was good--but what really MADE this place was the backyard dining area.")

f, p, a, s, r = nlp(text, aspects=['food', 'price','ambience','service', 'red'])
# assert price.sentiment == absa.Sentiment.negative
# assert slack.sentiment == absa.Sentiment.positive

In [18]:
# !pip install ipywidgets 
print(f.sentiment)
print(p.sentiment)
print(s.sentiment)
print(r.sentiment)

Sentiment.positive
Sentiment.negative
Sentiment.positive
Sentiment.positive


In [321]:
!which pip

/home/anoopkumar/anaconda3/envs/nlp/bin/pip
